<!-- # CNN autoencoder and Clustering from MTRX data

Use this notebook to load Scienta Omicron Matrix format SPM data and create standardised images for machine learning training and analysis. The code can generate both JPG image data, useful for manually checking the data, and windowed numpy data that can be loaded into ML models. 

The notebook then creates an autoencoder for training on a large dataset, followed by KMEANS clustering. 

**Author**: Steven R. Schofield  
**Created**: November, 2024 -->

# CASTEP - Si 001 surface
## Steven R. Schofield (Universtiy College London) May 2025

### Determine appropriate paths whether we are working on macbook or the cluster

In [1]:
import sys
from pathlib import Path

# Define candidate paths using Path objects
module_path_list = [
    Path('/Users/steven/academic-iCloud/Python/modules'),
    Path('/hpc/srs/Python/modules')
]

data_path_list = [
    Path('/Users/steven/academic-iCloud/Calculations/castep/'),
    Path('/hpc/srs/castep')
]

# Resolve actual paths
module_path = next((p for p in module_path_list if p.exists()), None)
data_path = next((p for p in data_path_list if p.exists()), None)

# Check and report missing paths
if module_path is None:
    print("Error: Could not locate a valid module path.")
if data_path is None:
    print("Error: Could not locate a valid data path.")

if module_path is None or data_path is None:
    sys.exit(1)

# Add module_path to sys.path if needed
if str(module_path) not in sys.path:
    sys.path.insert(0, str(module_path))

# Print resolved paths
print(f"module_path = {module_path}")
print(f"data_path = {data_path}")

module_path = /Users/steven/academic-iCloud/Python/modules
data_path = /Users/steven/academic-iCloud/Calculations/castep


### Import modules

In [2]:
# # Ensure modules are reloaded 
%load_ext autoreload
%autoreload 2

# Import standard modules
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt

# Import custom module
import SRSCALCUTILS.castep_tools as ct

from IPython.display import display

### Paths for the calculation

In [3]:
# General
job_folder = 'work-2025/si001_surf'
job_path = data_path / job_folder
verbose = False             # Set this True to print out more information

# Print resolved paths
print(f"job_path = {job_path}")

# If it doesn’t exist, create it (including any missing parent folders)
if not os.path.isdir(job_path):
    os.makedirs(job_path, exist_ok=True)

# Now safely switch into it
os.chdir(job_path)

print('Current files in this directory:')
for file in os.listdir():
    print(file)

job_path = /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf
Current files in this directory:
si001_test.cell
si001_test.param


### Base parameters of calculation and unit cell

In [4]:
# Set up the param and cell files for the calculation

filename = "si001_surf"
title = "silicon 001 surface tests"

params = {'task' : 'geomopt',
          'xc_functional'           : 'PBE',
          'cut_off_energy'          : 400,
          'spin_polarised'          : 'false',
          'write_cell_structure'    : 'true',
          'charge'                  : 0,
          'nextra_bands'            : 8,
          'geom_energy_tol'         : 0.00005,          # default: 0.00005 eV 
          'geom_disp_tol'           : 0.002,            # default: 0.002 Ang.
          'geom_max_iter'           : 100,              # default: 30
          'geom_convergence_win'    : 2,                # default: 2
          'max_scf_cycles'          : 300               # default: 30
          }              

lattice_cart = np.array([
            [3.8641976,     0.0,     0.0],
            [0.0,     3.8641976,     0.0],
            [0.0,     0.0,     5.4648012]
        ])

positions_frac = np.array([
            ['Si', 0.0,  0.0,  0.0],
            ['Si', 0.5,  0.0,  0.25],
            ['Si', 0.5,  0.5,  0.5],
            ['Si', 0.0,  0.5,  0.75],
        ])

cell_constraints = np.array([
            [0,  0,  0],
            [0,  0,  0]
        ])

kpoints_mp_grid = np.array([2, 2, 2])

fix_all_ions = False
symmetry_generate=False
symmetry_tol = 0.01


### Build supercell

In [5]:
n = [1, 2, 3] 
positions_frac, lattice_cart = ct.create_supercell_from_fractional_coords(positions_frac,lattice_cart,n)

### Add hydrogen termination to cell bottom 

In [6]:
# First select the atoms on the bottom layer 
axis = 'z'
criteria = "minimum"
ref_value, _ = ct.find_plane_value(positions_frac, lattice_cart, axis, criteria)
tolerance = 0.5
positions_frac = ct.select_atoms_by_plane(positions_frac, lattice_cart, axis, ref_value,
                          tolerance=tolerance, include=None, exclude=None)

# Next calculate the relative bond vectors for the two hydrogen atoms (per Si)
phi = np.deg2rad(90)    
si_h_bond_length = 1.5                         
theta_h_1 = np.deg2rad(180) - np.arccos(-1/3) / 2                
theta_h_2 = np.deg2rad(180) + np.arccos(-1/3) / 2            
bond_h_1  = ct.bond_vector_from_spherical(theta_h_1, phi, si_h_bond_length)                   
bond_h_2  = ct.bond_vector_from_spherical(theta_h_2, phi, si_h_bond_length)

# Now add the hydrogen atoms to the silicon atoms selected above using the calculated bond vectors
positions_frac, lattice_cart = ct.add_atoms_to_positions_frac(
                                    positions_frac,
                                    lattice_cart,
                                    [bond_h_1,bond_h_2],
                                    extend_unit_cell=(0, 0, 1),
                                    atom="H")
positions_frac = ct.sort_positions_frac(positions_frac)

In [7]:
# First select the atoms on the bottom layer 
axis = 'z'
criteria = "maximum"
ref_value, _ = ct.find_plane_value(positions_frac, lattice_cart, axis, criteria)
tolerance = 0.5
labelled_positions_frac = ct.select_atoms_by_plane(positions_frac, lattice_cart, axis, ref_value,
                          tolerance=tolerance, include=None, exclude=None)

# Next calculate the relative bond vectors for the two hydrogen atoms (per Si)
phi = np.deg2rad(90)    
si_h_bond_length = 1.5                         
theta_h_1 = np.deg2rad(360) - np.arccos(-1/3) / 2                
theta_h_2 = np.deg2rad(360) + np.arccos(-1/3) / 2            
bond_h_1  = ct.bond_vector_from_spherical(theta_h_1, phi, si_h_bond_length)                   
bond_h_2  = ct.bond_vector_from_spherical(theta_h_2, phi, si_h_bond_length)

# Now add the hydrogen atoms to the silicon atoms selected above using the calculated bond vectors
positions_frac, lattice_cart = ct.add_atoms_to_positions_frac(
                                    labelled_positions_frac,
                                    lattice_cart,
                                    [bond_h_1,bond_h_2],
                                    extend_unit_cell=(0, 0, 1),
                                    atom="H")
positions_frac = ct.sort_positions_frac(positions_frac)

### Add vacuum spacing

In [8]:
vac = 10
positions_frac, lattice_cart = ct.create_vacuum_spacing(positions_frac, lattice_cart, vac)

### Add ionic contraints

In [9]:
# Create fixed ion constraints
conditions = "z < 2.5"
ionic_constraints = ct.select_atoms_by_region(positions_frac, lattice_cart, conditions)

In [10]:
filename = f"si001_test"
castep_path = job_path / f"{filename}.castep"

print('=' * 80 + f'\nCASTEP file: {castep_path}\n' + '=' * 80 + '\n')   

param_filename = ct.write_param_file(
    params,
    title = title,
    filename=filename,
    path=job_path,
    display_file=True
    )

cell_filename = ct.write_cell_file(
    title = title,
    path=job_path,
    filename=filename,
    lattice_cart=lattice_cart,
    positions_frac=positions_frac,
    cell_constraints=cell_constraints,
    ionic_constraints=ionic_constraints,
    fix_all_ions=fix_all_ions,
    symmetry_generate=symmetry_generate,
    symmetry_tol = symmetry_tol,
    kpoints_mp_grid=kpoints_mp_grid,
    display_file=True
)

CASTEP file: /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf/si001_test.castep

Wrote param file to: /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf/si001_test.param
!TITLE: silicon 001 surface tests

TASK                 : geomopt
XC_FUNCTIONAL        : PBE
CUT_OFF_ENERGY       : 400
SPIN_POLARISED       : false
WRITE_CELL_STRUCTURE : true
CHARGE               : 0
NEXTRA_BANDS         : 8
GEOM_ENERGY_TOL      : 5e-05
GEOM_DISP_TOL        : 0.002
GEOM_MAX_ITER        : 100
GEOM_CONVERGENCE_WIN : 2
MAX_SCF_CYCLES       : 300


Wrote cell file to: /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf/si001_test.cell
! silicon 001 surface tests

%BLOCK lattice_cart
   ANG
       3.8641976000    0.0000000000    0.0000000000
       0.0000000000    7.7283952000    0.0000000000
       0.0000000000    0.0000000000   27.2604290038
%ENDBLOCK lattice_cart

%BLOCK CELL_CONSTRAINTS
       0    0    0
       0    0    0
%ENDBLOCK CELL_CO

In [11]:
# if str(module_path) == '/hpc/srs/Python/modules':
#     !bash -l -c "mpirun -np 62 castep.mpi {filename}"
# else:
#     !zsh -l -c "castepmpi {filename}"


In [12]:

# unit_cell, a, b, c, alpha, beta, gamma = ct.get_final_lattice_parameters(castep_path)
# energy_optimisation = ct.get_LBFGS_energies(castep_path)
# energies = [val for _, val in energy_optimisation]
# energy = energies[-1]

# print('Optimised energy {} eV'.format(energy))
# print('Optimised lattice constants a,b = {} Ang., c = {} Ang.'.format(a,c))
# print('Unit cell:')
# for line in unit_cell:
#     print(' '*2,np.abs(line))
